# 新聞情報取得

In [1]:
import os
import openpyxl

In [2]:
path_list = []

for curDir, dirs, files in os.walk("C:\\Users\\kenbun\\PycharmProjects\\ProtKenbunTalk\\Excel_News\\新聞"):
    #         print('=='*20)
    #         print('現在のフォルダ: ' + curDir)
    #         print('内包するフォルダ: ', end='')
    #         print(dirs)
    #         print('内包するファイル: ', end='')
    #         print(files)
    path = curDir + '\\'
    for file in files:
        path += file
        #             print('ファイルパス: ' + path)
        path_list.append(path)
        path = curDir + '\\'

In [3]:
path_list

['C:\\Users\\kenbun\\PycharmProjects\\ProtKenbunTalk\\Excel_News\\新聞\\見出し1.xlsx',
 'C:\\Users\\kenbun\\PycharmProjects\\ProtKenbunTalk\\Excel_News\\新聞\\見出し2.xlsx']

In [4]:
headlines = []  # 格納用辞書

wb1 = openpyxl.load_workbook(path_list[0])
sheet = wb1['シート1']
# [社名、年、月、名前、見出し、ベクトル]
headlines = [(row[0].value, row[1].value, row[2].value, row[3].value, row[4].value) for row in sheet.rows]

wb2 = openpyxl.load_workbook(path_list[1])
sheet = wb2['シート1']
headlines2 = [(row[0].value, row[1].value, row[2].value, row[3].value, row[4].value) for row in sheet.rows]

In [5]:
headlines += headlines2

In [6]:
len(headlines)

1253950

# 分かち書き用

In [7]:
import MeCab as mc
import mojimoji
m = mc.Tagger("-Ochasen")

In [8]:
def mecab_tokenizer(text: str):
    """
    テキストを分かち書きするメソッド
    :param text: 分割したいテキスト
    :return: 分割後のテキスト
    """

    node = m.parseToNode(text)
    word_list = list()
    while node:
        if node.surface != "":
            res = node.feature.split(",")
            word_type = res[0]
            if word_type in ['名詞', "動詞", "形容詞", "副詞"]:  # 名詞, 動詞, 形容詞, 副詞のみを抽出
                basic_word = res[6]
                if basic_word != "*":
                    word_list.append(basic_word)
                else:
                    word_list.append('[UNK]')  # 未知語の場合は[UNK]トークンに置き換え
        node = node.next
        if node is None:
            break
    return word_list

# Wordnetを利用
参考サイト: https://qiita.com/pocket_kyoto/items/f2382b9821d32624e6bc

In [9]:
import sqlite3
conn = sqlite3.connect("wnjpn.db")

In [10]:
class node:
    def __init__(self, name, children=None):
        self.name = name  # String
        self.children = children  # List of Class node

    # 結果表示用
    def display(self, indent = 0):
        if self.children != None:
            print(' '*indent + self.name)
            for c in self.children:
                c.display(indent+1)
        else:
            print(' '*indent + self.name)

In [11]:
# 上位-下位の関係にある概念の抽出
hierarchy_dict = {}  # key:上位語(String), value:下位語(List of String)
n_term_set = set()  # 下位語に含まれる単語集合

cur = conn.execute("select synset1,synset2 from synlink where link='hypo'")  # 上位語-下位語の関係にあるものを抽出
for row in cur:
    b_term = row[0]
    n_term = row[1]

    if b_term not in hierarchy_dict:
        hierarchy_dict[b_term] = []

    hierarchy_dict[b_term].append(n_term) 
    n_term_set.add(n_term)

print(f"上位語に含まれる単語の数 ： {len(hierarchy_dict)}")

top_concepts = list(set(hierarchy_dict.keys()) - n_term_set)
print(f"上位語に含まれる単語の中で下位語に含まれない単語の数 ： {len(top_concepts)}")

上位語に含まれる単語の数 ： 20008
上位語に含まれる単語の中で下位語に含まれない単語の数 ： 346


In [12]:
# synset(概念)のIDから、概念の名称に変換する辞書の作成
synset_name_dict = {}  # key:synsetのID, value:synsetの名称
cur = conn.execute("select synset,name from synset")
for row in cur:
    synset_name_dict[row[0]] = row[1]
for k,v in synset_name_dict.items():
    print(f"{k} : {v}")
    break

07125096-n : expletive


In [13]:
# 下位の語が未登録の場合、再帰的に呼ばれる関数
def recrusive_register(term, synset_name_dict, hierarchy_dict, node_tree_dict):
    for term in hierarchy_dict[term]:
        if term not in node_tree_dict:
            if term in hierarchy_dict:
                recrusive_register(term, synset_name_dict, hierarchy_dict, node_tree_dict)
                node_tree_dict[term] = node(synset_name_dict[term], [node_tree_dict[t] for t in hierarchy_dict[term]])
            else:
                node_tree_dict[term] = node(synset_name_dict[term])

# データ投入
node_tree_dict = {}
for k in top_concepts:  # 最上位の語を起点として木構造を作成
    recrusive_register(k, synset_name_dict, hierarchy_dict, node_tree_dict)
    node_tree_dict[k] = node(synset_name_dict[k], [node_tree_dict[term] for term in hierarchy_dict[k]])


In [14]:
node_tree_dict[top_concepts[10]].display()

work
 work_through
  whip_through
 muck_about
  puddle
 carpenter
 clerk
 page
 wait
 pull_one's_weight
 electioneer
 assist
  protect
  abet
 beaver
 work_at
  belabor
 keep_one's_shoulder_to_the_wheel
 occupy
  potter
  play_around
 collaborate
  collaborate
  go_along
 volunteer
 specialize
 minister
 intern
 skipper
 boondoggle
 skimp
 whore
 serve
 work


In [15]:
# 下位-上位の関係にある概念の抽出
cur = conn.execute("select synset1,synset2 from synlink where link='hypo'")

hierarchy_dict = {}  # key:下位語(String), value:上位語(String)

for row in cur:
    b_term = row[0]
    n_term = row[1]

    if n_term not in hierarchy_dict:
        hierarchy_dict[n_term] = b_term

In [16]:
# 特定の単語を入力とした時に、上位語を検索する関数
def search_top_concept_words(word, hierarchy_dict):

    # 問い合わせしたい単語がWordnetに存在するか確認する
    cur = conn.execute(f"select wordid from word where lemma='{word}'")
    word_id = 99999999  #temp 
    for row in cur:
        word_id = row[0]

    # Wordnetに存在する語であるかの判定
    if word_id==99999999:
        print(f"「{word}」は、Wordnetに存在しない単語です。")
        return
    else:
        print(f"【「{word}」の最上位概念を出力します】\n")

    # 入力された単語を含む概念を検索する
    cur = conn.execute(f"select synset from sense where wordid='{word_id}'")
    synsets = []
    for row in cur:
        synsets.append(row[0])

    # 概念に含まれる単語を検索して画面出力する
    no = 1
    for synset in synsets:
        cur1 = conn.execute(f"select name from synset where synset='{synset}'")
        for row1 in cur1:
            print(f"{no}つめの概念 : {row1[0]}")
        cur2 = conn.execute(f"select def from synset_def where (synset='{synset}' and lang='jpn')")
        sub_no = 1
        for row2 in cur2:
            print(f"意味{sub_no} : {row2[0]}")
            sub_no += 1

        # 上位語の検索部分
        b_term = ""
        while(synset in hierarchy_dict.keys()):
            synset = hierarchy_dict[synset]

        cur1 = conn.execute(f"select name from synset where synset='{synset}'")
        for row1 in cur1:
            print(f"最上位概念 : {row1[0]}")

        cur2 = conn.execute(f"select def from synset_def where (synset='{synset}' and lang='jpn')")
        sub_no = 1
        for row2 in cur2:
            print(f"意味{sub_no} : {row2[0]}")
            sub_no += 1

        # 更新          
        print("\n")
        no += 1

In [17]:
test = []
for num, headline in enumerate(headlines):
    if num < 11:
        print('==' * 10)
        print(f'情報: {headline[0]}, {headline[1]}, {headline[2]}, {headline[3]}')
        print(f'見出し:\n{headline[4]}')
        test.append(headline[4]);

情報: こども大分新聞, 1932年, 10月, 2-y-2_01
見出し:
んぐみ
| day
三人組の隨一 ジ一堅い信念を貫いた。
BEATION FLOR ANATUREHOU
||||
情報: こども大分新聞, 1932年, 10月, 2-y-2_02
見出し:
四戒を守って 健やか&伸びる 專科に運動に大いに養
日田男子校の巻
情報: こども大分新聞, 1932年, 10月, 2-y-2_03
見出し:
少年 少 女 講 座 メートル法の起源
情報: こども大分新聞, 1932年, 10月, 2-y-9_01
見出し:
10WH
||||||||RUNWHIEF
電器具と 附屬品のいろく
文明國の小國民は
是非知って置きませう
情報: こども大分新聞, 1932年, 10月, 2-y-9_04
見出し:
hateway.
いろいろ
電燈線の
情報: こども大分新聞, 1932年, 10月, 2-y-4_01
見出し:
日本は殊に 霞が招はい それは気温を 氣壁の配置から」
情報: こども大分新聞, 1932年, 10月, 2-a-8_01
見出し:
の 發 明
情報: こども大分新聞, 1932年, 10月, 2-a-8_03
見出し:
でんわ
くった
電話機の組立
情報: こども大分新聞, 1932年, 10月, 2-y-6_02
見出し:
たましひ
1日本魂の權化
―肉弾三勇士の少年時代―
ゆう
情報: こども大分新聞, 1932年, 10月, 2-y-9_02
見出し:
照渡自在な。 スウィッチー
その變り種のいろく
情報: こども大分新聞, 1932年, 10月, 1-y-2_01
見出し:
おとな
・大人も及ば。
非凡な度胸と動


In [18]:
test

['んぐみ\n| day\n三人組の隨一 ジ一堅い信念を貫いた。\nBEATION FLOR ANATUREHOU\n||||',
 '四戒を守って 健やか&伸びる 專科に運動に大いに養\n日田男子校の巻',
 '少年 少 女 講 座 メートル法の起源',
 '10WH\n||||||||RUNWHIEF\n電器具と 附屬品のいろく\n文明國の小國民は\n是非知って置きませう',
 'hateway.\nいろいろ\n電燈線の',
 '日本は殊に 霞が招はい それは気温を 氣壁の配置から」',
 'の 發 明',
 'でんわ\nくった\n電話機の組立',
 'たましひ\n1日本魂の權化\n―肉弾三勇士の少年時代―\nゆう',
 '照渡自在な。 スウィッチー\nその變り種のいろく',
 'おとな\n・大人も及ば。\n非凡な度胸と動']

In [19]:
search_top_concept_words("ネコ", hierarchy_dict)

【「ネコ」の最上位概念を出力します】

1つめの概念 : true_cat
意味1 : 通常、厚く柔らかい毛皮を持ち、吠えることのできないネコ科の哺乳類：家ネコ
意味2 : ヤマネコ
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か




In [20]:
search_top_concept_words("月", hierarchy_dict)

【「月」の最上位概念を出力します】

1つめの概念 : moonlight
意味1 : 月の光
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か


2つめの概念 : month
意味1 : 暦年を12に分けた中の1つ
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か


3つめの概念 : month
意味1 : 約30日間という時間単位
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か


4つめの概念 : moon
意味1 : 地球の天然衛星
最上位概念 : moon
意味1 : 地球の天然衛星




In [21]:
search_top_concept_words(test[2], hierarchy_dict)

「少年 少 女 講 座 メートル法の起源」は、Wordnetに存在しない単語です。


In [22]:
splited_word = mecab_tokenizer(test[2])

In [23]:
splited_word

['少年', '女', '講', '座', 'メートル法', '起源']

In [24]:
for word in splited_word:
    print('==' * 10)
    print(f'単語: {word}')
    search_top_concept_words(word, hierarchy_dict)

単語: 少年
【「少年」の最上位概念を出力します】

1つめの概念 : blighter
意味1 : 少年または男
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か


2つめの概念 : juvenile
意味1 : 若い人
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か


3つめの概念 : male_child
意味1 : 若い男
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か


4つめの概念 : cub
意味1 : 男の子供（男の子への親しみを込めた呼びかけ語）
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か


5つめの概念 : teenager
意味1 : 思春期の入り口と成年の間にいる若者
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か


単語: 女
【「女」の最上位概念を出力します】

1つめの概念 : girl
意味1 : 親しい成人女性に対するくだけた呼びかた
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か


2つめの概念 : married_woman
意味1 : 結婚している女性
意味2 : 結婚における男性のパートナー
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か


3つめの概念 : female
意味1 : 子供を産むことのできる性に属する人
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身

In [25]:
# 新聞見出しのリストの容量
headlines.__sizeof__()
# 11285632 -> 1.12GB

11285632